In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm import tqdm

%matplotlib inline

PATH = '../'

In [2]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'Track 1/test.csv')
submission = pd.read_csv(PATH + 'Track 1/submission.csv')

Wall time: 3.26 s


### LB Score Calc

Для подсчета скора просто используйте функцию getLBScore

In [3]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs(satellite_predicted_values - satellite_true_values) / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values)))

### Model training

In [4]:
training_columns = ['epoch', 'sat_id', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
prediction_columns = ['x','y','z','Vx', 'Vy', 'Vz']
X = train.loc[:, training_columns]
y = train.loc[:, prediction_columns]
Xtest = test.loc[:, training_columns]

In [5]:
def extractTimeFeats(data):
    timeFeats = data['epoch'].apply(lambda x: str(x).split('T')[1].split('.')[0].split(':')).values
    h,m,s = [],[],[]
    for feat in timeFeats:
        h.append(feat[0]); m.append(feat[1]); s.append(feat[2])
    return h,m,s

In [6]:
h,m,s = extractTimeFeats(X)
X.loc[:, 'h'] = list(map(int, h)); X.loc[:, 'm'] = list(map(int, m)); X.loc[:, 's'] =list(map(int, s))
X.drop(columns = 'epoch', inplace = True)

h,m,s = extractTimeFeats(Xtest)
Xtest.loc[:, 'h'] = list(map(int, h)); Xtest.loc[:, 'm'] = list(map(int, m)); Xtest.loc[:, 's'] = list(map(int, s))
Xtest.drop(columns = 'epoch', inplace = True)

In [15]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
model = ExtraTreesRegressor()
trainSize = 0.8
submissionValid = train.copy()[['id', 'sat_id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']]
smp = []
nTargets = 6
for sat_id in Xtest['sat_id'].unique():
    satX = X[X['sat_id'] == sat_id]
    satY = y[X['sat_id'] == sat_id]
    satXtest = Xtest[Xtest['sat_id'] == sat_id]
    
    size = int(satX.shape[0] * trainSize)
    Xtr, Xval = satX.iloc[:size, :], satX.iloc[size:, :]
    ytr, yval = satY.iloc[:size, :], satY.iloc[size:, :]
    for i in range(nTargets):
        model.fit(Xtr, ytr.iloc[:, i])
        ypred = model.predict(Xval)
        smp.append(smape(ypred, yval.iloc[:, i]))
    print(np.mean(smp))
print('Final', 100*(1- np.mean(smp)))

KeyboardInterrupt: 

In [11]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import AdaBoostRegressor
model = ExtraTreesRegressor()
nTargets = 6
for sat_id in tqdm(Xtest['sat_id'].unique()):
    satX = X[X['sat_id'] == sat_id].drop(columns = ['sat_id'])
    satY = y[X['sat_id'] == sat_id]
    satXtest = Xtest[Xtest['sat_id'] == sat_id].drop(columns = ['sat_id'])
    
    for i in range(nTargets):
        model.fit(satX, satY.iloc[:, i])
        ypred = model.predict(satXtest)
        submission.loc[satXtest.index, submission.columns[i+1]] = ypred


  0%|                                                                                                                                                                                                                  | 0/300 [00:00<?, ?it/s]C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

 11%|██████████████████████                                                                                                                                                                                   | 33/300 [00:07<01:19,  3.35it/s]


 22%|████████████████████████████████████████████▉                                                                                                                                                            | 67/300 [00:13<00:49,  4.71it/s]


 34%|████████████████████████████████████████████████████████████████████                                                                                                                                    | 102/300 [00:22<00:59,  3.34it/s]


 45%|██████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                             | 136/300 [00:30<00:33,  4.93it/s]


 58%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                    | 173/300 [00:38<00:27,  4.66it/s]


 69%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                             | 208/300 [00:47<00:21,  4.18it/s]


 81%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                     | 244/300 [00:56<00:14,  3.77it/s]


 93%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████              | 279/300 [01:04<00:02,  7.38it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:08<00:00,  4.50it/s]

In [16]:
submission.to_csv('submission.csv', index = None)